# Unscramble

Unscramble wykorzystuje **DataStore**, aby zapisywać i odczytywać stan gry. Jest to mechanizm do przechowywania danych, który oferuje uproszczoną i bezpieczną alternatywę dla SharedPreferences.

Aplikacja została zaimplementowana zgodnie z wzorcem **MVVM (Model-View-ViewModel)** z wykorzystaniem **repozytorium**. Wzorzec ten pomaga oddzielić logikę biznesową od warstwy prezentacji, co ułatwia testowanie, utrzymanie i rozwijanie aplikacji.

Do zarządzania stanem aplikacji i reaktywności został użyty **StateFlow**, umożliwiając łatwą aktualizację widoków w czasie rzeczywistym w odpowiedzi na zmiany w stanie gry.

Aplikacja bazuje na oficjalnym [Android Codelab](https://developer.android.com/codelabs/basic-android-kotlin-training-viewmodel#0)

<table><tr><td><img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExYWwxOGlnNGQ3Mm9xOGRraHFuaXFyOXA0dXkwNWI2OGx0M2RvbndlMSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c4MVddvPAaRxrwp9JB/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMnVzODY5dDR1a2QxdDk0MmtibWhmZXM0ZW0yc2VkcWZwd3NkaHlmZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/t3ERZ9ZxBVMeNiY1KY/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExbW8yN201Y3EzOHN6MGZsenhlMTM1ZHU0dWY5bmtkd2lvcTVybnZyZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/bCYIyjvwkKXnvrGd3n/giphy.gif" width="200" /></td></tr></table>

- Aplikacja zaimplementowana zgodnie ze wzoccem MVVM wraz z repozytorium
- `StateFlow` został wykorzystany do łatwego zarządzania bieżącym stanem interfejsu użytkownika
- Klasa `GameUiState` modeluje stan interfejsu użytkownika
- Klasa `GameState` modeluje stan samej gry
- `DataStore` został wykorzystany do zapisu i odczyta stanu gry
- Repozytorium stanowi warstwę pomiędzy `GameStateDataStore` a `GameViewModel`, zapewniając izolację odpowiedzialności i umożliwiając łatwy dostęp do operacji na pliku `DataStore`


Dodajmy wymagane zależności do projektu

```kotlin
    // ViewModel
    implementation ("androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.1")
    //Fragment
    implementation ("androidx.fragment:fragment-ktx:1.6.1")

    //DataStore
    implementation ("androidx.datastore:datastore-preferences:1.0.0")
```

Aplikacja będzie pozwalać zgadywać słowa, więc potrzebujemy listę słów do użycia.

In [ ]:
const val MAX_NO_OF_WORDS = 10
const val SCORE_INCREASE = 20

object DataProvider {
    val words = listOf(
        "kot", "pies", "drab", "kawa", "samolot", "wino", "gracz", "woda", "dom",
        "karma", "tanie", "bilet", "muzyk", "rybak", "chleb", "motyw",
        "las", "papuga", "talerz", "stacja", "grupa", "butelka", "kurczak",
        "okno", "drzewo", "lampa", "sklep", "kasa", "broda", "papier", "szafa",
        "dzwon", "kotek"
    )
}

Gra będzie składała się z próby odgadnięcia 10 słów, za każde poprawne słowo gracz otrzymuje 20 punktów. W kolejnym kroku przygotujmy klasę `GameUiState` zawierającą wszystkie informacje, które chcemy pokazać użytkownikowi.

In [ ]:
data class GameUiState(
    val currentScrambledWord: String = "",
    val currentWordCount: Int = 1,
    val score: Int = 0,
    val isGuessedWordWrong: Boolean = false,
    val isGameOver: Boolean = false
)

- `currentScrambledWord: String = ""` - aktualnie wyświetlane słowo
- `currentWordCount: Int = 1` - licznik słów
- `score: Int = 0` - wynik
- `isGuessedWordWrong: Boolean = false` - flaga pomocnicza określająca czy użytkownik poprawni odggadł słowo
- `isGameOver: Boolean = false` - flaga pomocnicza określająca czy gra powinna zostać zakończona

### ViewModel

Dodajmy klasę `GameViewModel`, klasa będzie zawierała kilka metod i pól:

- `uiState: StateFlow<GameUiState>` Jest to obiekt reprezentujący bieżący stan interfejsu użytkownika (UI). Składa się z obiektu typu GameUiState, który zawiera informacje o stanie gry na poziomie interfejsu użytkownika.
- `userGuess` Jest to zmienna, która przechowuje aktualne odgadnięte słowo wprowadzone przez użytkownika.
- `usedWords: MutableSet<String>` Jest to zbiór słów użytych w grze. Jest to wykorzystywane w celu śledzenia, które słowa już zostały wykorzystane.
- `currentWord: String` Zmienna przechowująca aktualne słowo, które jest obecnie odgrywane w grze.
- `saveGame()` Funkcja służąca do zapisywania stanu gry. Wywołuje funkcję `saveGameState` z repozytorium, aby zapisać bieżący stan` GameUiState`, `usedWords` oraz `currentWord`.
- `loadGame()` Funkcja służąca do wczytywania stanu gry.
- `resetGame()` Funkcja inicjująca grę od nowa. Czyści zbiór `usedWords` i ustawia nowe, losowe słowo jako bieżące słowo w `uiState`.
- `updateUserGuess(guessedWord: String)` Funkcja aktualizująca odgadnięte słowo użytkownika (userGuess) na podstawie wprowadzonego przez niego słowa.
- `checkUserGuess()` Funkcja sprawdzająca, czy odgadnięte słowo użytkownika (`userGuess`) jest poprawne. W zależności od wyniku, funkcja zwiększa wynik (`score`) i przygotowuje grę na kolejną rundę.
- `skipWord()` Funkcja pozwalająca na pominięcie obecnego słowa i przechodzenie do następnego słowa.
- `updateGameState(updatedScore: Int)` Funkcja, która aktualizuje stan gry (`GameUiState`). Decyduje, czy gra poinna zostać zakończona, czy też powinna przejść do kolejnej rundy.
- `shuffleCurrentWord(word: String): String` Funkcja przetasowująca litery w podanym słowie i zwracająca przetasowane słowo jako wynik.
- `pickRandomWordAndShuffle(): String` Funkcja wybierająca losowe słowo z dostępnych słów w `DataProvider` i przetasowująca je za pomocą `shuffleCurrentWord()`. Funkcja zapewnia, że wybrane słowo nie zostało użyte wcześniej w grze, korzystając z zbioru `usedWords`.

W pierwszym kroku dodajemy instancję `GameUiState`

In [ ]:
private val _uiState = MutableStateFlow(GameUiState())
val uiState: StateFlow<GameUiState> = _uiState.asStateFlow()

Następnie musimy sledzić aktualnie wpisane słowo przez użytkownika.

In [ ]:
var userGuess by mutableStateOf("")
    private set

Potrzebujemy jeszcze dwie zmienne.

In [ ]:
private var usedWords: MutableSet<String> = mutableSetOf()
private lateinit var currentWord: String

`usedWords` śledzące wszystkie słowa, które zostały już wylosowane z listy dostępnej w `DataProvider`, oraz `currentWord` przechowującą aktualnie wylosowane słowo.

Dodajmy funkcję resetującą całą grę.

In [ ]:
fun resetGame() {
    usedWords.clear()
    _uiState.value = GameUiState(currentScrambledWord = pickRandomWordAndShuffle())
}

- `usedWords.clear()` - czyści zbiór wykorzystanych słów
- `_uiState.value = GameUiState(currentScrambledWord = pickRandomWordAndShuffle())` - inicjuje `_uiState` domyślnymi wartościami, oraz losuje aktualne słowo.

Wywołajmy funkcję `reset` w bloku `init`, inicjując w ten sposób grę.

In [ ]:
init {
    resetGame()
}

Dodajmy funkcję `pickRandomWordAndShuffle()`, która losowo wybiera słowo z dostępnej listy oraz losowo przestawia jego litery.

In [ ]:
private fun pickRandomWordAndShuffle(): String {
    // Continue picking up a new random word until you get one that hasn't been used before
    currentWord = DataProvider.words.random()
    return if (usedWords.contains(currentWord)) {
        pickRandomWordAndShuffle()
    } else {
        usedWords.add(currentWord)
        shuffleCurrentWord(currentWord)
    }
}

Wybieramy nowe słowo, dopóki funkcja nie zwróci wcześniej nieużytego. Po wyborze dodajemy nowe słowo do zbioru już użytych i wykonujemy wymieszanie liter.

In [ ]:
private fun shuffleCurrentWord(word: String): String {
    val tempWord = word.toCharArray()
    // Scramble the word
    tempWord.shuffle()
    while (String(tempWord) == word) {
        tempWord.shuffle()
    }
    return String(tempWord)
}

Na początku, podane słowo word jest zamieniane na tablicę znaków za pomocą metody `toCharArray()`. Dzięki temu możemy łatwo manipulować poszczególnymi literami słowa. Następnie wywołujemy funkcję `shuffle()` na tablicy. Metoda losowo przestawia elementy tablicy. Pętla `while` jest użyta, aby zapewnić, że przetasowanie słowa będzie różne od początkowego słowa.

W kolejnym kroku zaimplementujmy prywatną funkcję `updateGameState`, która jest odpowiedzialna za aktualizację stanu gry. Funkcja ta decyduje, czy gra się zakończyła, czy powinna przejść do kolejnej rundy.

In [ ]:
private fun updateGameState(updatedScore: Int) {
    if (usedWords.size == MAX_NO_OF_WORDS){
        //Last round in the game, update isGameOver to true, don't pick a new word
        _uiState.update { currentState ->
            currentState.copy(
                isGuessedWordWrong = false,
                score = updatedScore,
                isGameOver = true
            )
        }
    } else{
        // Normal round in the game
        _uiState.update { currentState ->
            currentState.copy(
                isGuessedWordWrong = false,
                currentScrambledWord = pickRandomWordAndShuffle(),
                currentWordCount = currentState.currentWordCount.inc(),
                score = updatedScore
            )
        }
    }
}

Wpierw sprawdzamy, czy liczba użytych słów jest równa maksymalnej liczbie słów dozwolonej w grze. Jeśli ten warunek jest spełniony, oznacza to, że gra osiągnęła ostatnią rundę. 

`_uiState` jest aktualizowany za pomocą funkcji `update`. Tworzy ona nowy stan `GameUiState`, ale z niektórymi właściwościami zmienionymi.

Funkcja `update` aktualizuje wartość `MutableStateFlow` **atomowo** - wykonywane jest jako pojedyncza, niepodzielna i niezmienna operacja, oznacza to, że operacja atomowa jest wykonana w całości lub w ogóle, bez możliwości przerwania lub częściowego wykonania przez inne wątki lub procesy.

Następnie zaimplementujmy metodę aktualizującą słowo wprowadzone przez użytkownika

In [ ]:
fun updateUserGuess(guessedWord: String){
    userGuess = guessedWord
}

oraz metodę odpowiedzialną za sprawdzenie, czy słwowo wprowadzone przez użytkownika jest prawidłowe

In [ ]:
fun checkUserGuess() {
    if (userGuess.equals(currentWord, ignoreCase = true)) {
        // User's guess is correct, increase the score
        // and call updateGameState() to prepare the game for next round
        val updatedScore = _uiState.value.score.plus(SCORE_INCREASE)
        updateGameState(updatedScore)
    } else {
        // User's guess is wrong, show an error
        _uiState.update { currentState ->
            currentState.copy(isGuessedWordWrong = true)
        }
    }
    // Reset user guess
    updateUserGuess("")
}

Jeżeli użytkownik podał prawidłowe słowo, aktualizujemy wynik, oraz wywołujemy funkcję `updateGameState` aby przejść do kolejnej rundy. Jeżeli wprowadzone słowo jest nieprawidłowe, zmieniamy flagę `isGuessedWordWrong` na `true`. Na konie resetujemy zmienną `userGuess`.

### Zapis do pliku

Pozostało dodanie funkcjonalności zapisu i odczytu z pliku. Rozpocznijmy od dodania klasy `GameState`, która przechowuje wszystkie niezbędne informacje o stanie gry.

In [ ]:
data class GameState (
    val gameUiState: GameUiState,
    val usedWords: Set<String>,
    val currentWord: String
)

Następnie dodajmy klasę pomocniczą `GameStateDataStore`

In [ ]:
object GameStateDataStore {

    private val Context.dataStore: DataStore<Preferences> by preferencesDataStore("game_state")

    private val CURRENT_SCRAMBLED_WORD_KEY = stringPreferencesKey("current_scrambled_word")
    private val CURRENT_WORD_COUNT_KEY = intPreferencesKey("current_word_count")
    private val SCORE_KEY = intPreferencesKey("score")
    private val IS_GUESSED_WORD_WRONG_KEY = booleanPreferencesKey("is_guessed_word_wrong")
    private val IS_GAME_OVER_KEY = booleanPreferencesKey("is_game_over")
    private val USED_WORDS_KEY = stringSetPreferencesKey("used_words")
    private val CURRENT_WORD_KEY = stringPreferencesKey("current_word")

    suspend fun saveGameState(context: Context, gameState: GameState) {
        context.dataStore.edit { preferences ->
            preferences[CURRENT_SCRAMBLED_WORD_KEY] = gameState.gameUiState.currentScrambledWord
            preferences[CURRENT_WORD_COUNT_KEY] = gameState.gameUiState.currentWordCount
            preferences[SCORE_KEY] = gameState.gameUiState.score
            preferences[IS_GUESSED_WORD_WRONG_KEY] = gameState.gameUiState.isGuessedWordWrong
            preferences[IS_GAME_OVER_KEY] = gameState.gameUiState.isGameOver
            preferences[USED_WORDS_KEY] = gameState.usedWords
            preferences[CURRENT_WORD_KEY] = gameState.currentWord
        }
    }

    fun loadGameState(context: Context): Flow<GameState> {
        return context.dataStore.data
            .catch { exception ->
                // DataStore throws an IOException if an error is encountered when reading data
                if (exception is IOException) {
                    emit(emptyPreferences())
                } else {
                    throw exception
                }
            }
            .map { preferences ->
                val currentScrambledWord = preferences[CURRENT_SCRAMBLED_WORD_KEY] ?: ""
                val currentWordCount = preferences[CURRENT_WORD_COUNT_KEY] ?: 1
                val score = preferences[SCORE_KEY] ?: 0
                val isGuessedWordWrong = preferences[IS_GUESSED_WORD_WRONG_KEY] ?: false
                val isGameOver = preferences[IS_GAME_OVER_KEY] ?: false
                val usedWordsSet = preferences[USED_WORDS_KEY] ?: emptySet()
                val currentWord = preferences[CURRENT_WORD_KEY] ?: ""

                GameState(
                    GameUiState(
                        currentScrambledWord,
                        currentWordCount,
                        score,
                        isGuessedWordWrong,
                        isGameOver
                    ),
                    usedWordsSet.toMutableSet(),
                    currentWord
                )
            }
    }
}

- `private val ..._KEY` Definicje kluczy, które będą używane do przechowywania poszczególnych elementów stanu gry w `DataStore`.
- `suspend fun saveGameState(context: Context, gameState: GameState)` Funkcja do zapisywania stanu gry.
- `fun loadGameState(context: Context): Flow<GameState>` Funkcja do wczytywania stanu gry. Zwraca obiekt typu `Flow<GameState>`, który emituje strumień wartości stanu gry.

Dodajmy repozytorium.

In [ ]:
class GameRepository(private val application: Application) {
    fun loadGameState() = GameStateDataStore.loadGameState(application)
    suspend fun saveGameState(state: GameState) = GameStateDataStore.saveGameState(application, state)
}

Ponieważ `DataStore` wymaga kontekstu, musimy wykorzystać `AndroidViewModel`, tak jak w poprzednich przykładach.

Powróćmy do klasy `GameViewModel` i dodajmy metody odpowiedzialne za zapis i odczyt stanu gry.

In [ ]:
fun saveGame(){
    viewModelScope.launch {
        repository.saveGameState(GameState(
            gameUiState = _uiState.value,
            usedWords = usedWords,
            currentWord = currentWord
        ))
    }
}

fun loadGame(){
    viewModelScope.launch {
        repository.loadGameState().collect{ gameState ->
            // odczytujemy zapisany stan interfejsu użytkownika i aktualizujemy zmienną _uiState
            _uiState.update { currentState ->
                currentState.copy(
                    currentScrambledWord = gameState.gameUiState.currentScrambledWord,
                    currentWordCount = gameState.gameUiState.currentWordCount,
                    score = gameState.gameUiState.score,
                    isGuessedWordWrong = gameState.gameUiState.isGuessedWordWrong,
                    isGameOver = gameState.gameUiState.isGameOver
                )
            }
            
            // odczytujemy wykorzystane w grze słowa
            usedWords = gameState.usedWords.toMutableSet()
            
            // odczytujemy ostatnie wylosowane słowo
            currentWord = gameState.currentWord
        }
    }
}

Pełny kod `GameViewModel`

In [ ]:
class GameViewModel(application: Application) : AndroidViewModel(application) {

    private val repository: GameRepository

    // Game UI state
    private val _uiState = MutableStateFlow(GameUiState())
    val uiState: StateFlow<GameUiState> = _uiState.asStateFlow()

    var userGuess: String = ""
        private set

    // Set of words used in the game
    private var usedWords: MutableSet<String> = mutableSetOf()
    private lateinit var currentWord: String

    init {
        repository = GameRepository(application)
        resetGame()
    }

    fun saveGame(){
        viewModelScope.launch {
            repository.saveGameState(
                GameState(
                gameUiState = _uiState.value,
                usedWords = usedWords,
                currentWord = currentWord
            )
            )
        }
    }

    fun loadGame(){
        viewModelScope.launch {
            repository.loadGameState().collect{ gameState ->
                _uiState.update { currentState ->
                    currentState.copy(
                        currentScrambledWord = gameState.gameUiState.currentScrambledWord,
                        currentWordCount = gameState.gameUiState.currentWordCount,
                        score = gameState.gameUiState.score,
                        isGuessedWordWrong = gameState.gameUiState.isGuessedWordWrong,
                        isGameOver = gameState.gameUiState.isGameOver
                    )
                }
                usedWords = gameState.usedWords.toMutableSet()
                currentWord = gameState.currentWord
            }
        }
    }

    /*
     * Re-initializes the game data to restart the game.
     */
    fun resetGame() {
        usedWords.clear()
        _uiState.value = GameUiState(currentScrambledWord = pickRandomWordAndShuffle())
    }

    /*
     * Update the user's guess
     */
    fun updateUserGuess(guessedWord: String){
        userGuess = guessedWord
    }

    /*
     * Checks if the user's guess is correct.
     * Increases the score accordingly.
     */
    fun checkUserGuess() {
        if (userGuess.equals(currentWord, ignoreCase = true)) {
            // User's guess is correct, increase the score
            // and call updateGameState() to prepare the game for next round
            val updatedScore = _uiState.value.score.plus(SCORE_INCREASE)
            updateGameState(updatedScore)
        } else {
            // User's guess is wrong, show an error
            _uiState.update { currentState ->
                currentState.copy(isGuessedWordWrong = true)
            }
        }
        // Reset user guess
        updateUserGuess("")
    }

    /*
     * Skip to next word
     */
    fun skipWord() {
        updateGameState(_uiState.value.score)
        // Reset user guess
        updateUserGuess("")
    }

    /*
     * Picks a new currentWord and currentScrambledWord and updates UiState according to
     * current game state.
     */
    private fun updateGameState(updatedScore: Int) {
        if (usedWords.size == MAX_NO_OF_WORDS){
            //Last round in the game, update isGameOver to true, don't pick a new word
            _uiState.update { currentState ->
                currentState.copy(
                    isGuessedWordWrong = false,
                    score = updatedScore,
                    isGameOver = true
                )
            }
        } else{
            // Normal round in the game
            _uiState.update { currentState ->
                currentState.copy(
                    isGuessedWordWrong = false,
                    currentScrambledWord = pickRandomWordAndShuffle(),
                    currentWordCount = currentState.currentWordCount.inc(),
                    score = updatedScore
                )
            }
        }
    }

    private fun shuffleCurrentWord(word: String): String {
        val tempWord = word.toCharArray()
        // Scramble the word
        tempWord.shuffle()
        while (String(tempWord) == word) {
            tempWord.shuffle()
        }
        return String(tempWord)
    }

    private fun pickRandomWordAndShuffle(): String {
        // Continue picking up a new random word until you get one that hasn't been used before
        currentWord = DataProvider.words.random()
        return if (usedWords.contains(currentWord)) {
            pickRandomWordAndShuffle()
        } else {
            usedWords.add(currentWord)
            shuffleCurrentWord(currentWord)
        }
    }
}

### Interfejs użytkownika

Ostatnim elementem będzie utworzenie fragmentu, rozpocznijmy od layotu.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<ScrollView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <androidx.constraintlayout.widget.ConstraintLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:padding="8dp"
        tools:context=".ui.game.GameFragment">

        <Button
            android:id="@+id/skip"
            style="?attr/materialButtonOutlinedStyle"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_marginStart="8dp"
            android:layout_marginEnd="8dp"
            android:text="Pomiń"
            app:layout_constraintBaseline_toBaselineOf="@+id/submit"
            app:layout_constraintEnd_toStartOf="@+id/submit"
            app:layout_constraintStart_toStartOf="parent" />

        <Button
            android:id="@+id/submit"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_marginTop="8dp"
            android:text="Sprawdź"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toEndOf="@+id/skip"
            app:layout_constraintTop_toBottomOf="@+id/textField" />

        <Button
            android:id="@+id/save"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_margin="8dp"
            android:text="Zapisz"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/skip" />

        <Button
            android:id="@+id/load"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_margin="8dp"
            android:text="Wczytaj"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/save" />

        <TextView
            android:id="@+id/textView_unscrambled_word"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginTop="8dp"
            android:layout_marginBottom="8dp"
            android:text="Unscramble"
            android:textAppearance="@style/TextAppearance.MaterialComponents.Headline3"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/title"
            tools:text="Unscramble" />

        <TextView
            android:id="@+id/title"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginTop="20dp"
            android:layout_marginBottom="8dp"
            android:text="Unscramble"
            android:textAppearance="@style/TextAppearance.MaterialComponents.Headline4"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/word_count"
            tools:text="Unscramble" />

        <TextView
            android:id="@+id/word_count"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="licnzik"
            android:textAppearance="@style/TextAppearance.MaterialComponents.Headline6"
            app:layout_constraintBottom_toTopOf="@+id/title"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent"
            tools:text="3 of 10 words" />

        <TextView
            android:id="@+id/score"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="Wynik"
            android:textAllCaps="true"
            android:textAppearance="@style/TextAppearance.MaterialComponents.Headline6"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintTop_toTopOf="parent"
            tools:text="Score: 20" />

        <com.google.android.material.textfield.TextInputLayout
            android:id="@+id/textField"
            android:layout_width="0dp"
            android:layout_height="wrap_content"
            android:layout_marginTop="8dp"
            android:hint="Zgadnij"
            app:errorIconDrawable="@drawable/ic_error"
            app:helperTextTextAppearance="@style/TextAppearance.MaterialComponents.Subtitle1"
            app:layout_constraintBottom_toTopOf="@+id/submit"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/textView_unscrambled_word">

            <com.google.android.material.textfield.TextInputEditText
                android:id="@+id/userInput"
                android:layout_width="match_parent"
                android:layout_height="match_parent"
                android:inputType="textPersonName|textNoSuggestions"
                android:maxLines="1" />
        </com.google.android.material.textfield.TextInputLayout>

    </androidx.constraintlayout.widget.ConstraintLayout>
</ScrollView>

Dodajmy samą klasę

In [ ]:
class UnscrambleFragment : Fragment() {

    private lateinit var binding: FragmentUnscrambleBinding

    private val viewModel: GameViewModel by viewModels()
    
    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentUnscrambleBinding.inflate(inflater)

        viewLifecycleOwner.lifecycleScope.launch {
            viewModel.uiState.collectLatest { uiState ->
                binding.wordCount.text = "${uiState.currentWordCount}/$MAX_NO_OF_WORDS"
                binding.textViewUnscrambledWord.text = uiState.currentScrambledWord
                binding.score.text = uiState.score.toString()

                if (uiState.isGameOver)
                    showFinalScoreDialog(uiState.score) // po zakończeniu gry pokazujemy alertDialog z podsumowaniem
            }
        }

        binding.apply {
            skip.setOnClickListener { viewModel.skipWord() } // onclick przycisku umożliwiającego pominięcie słowa
            submit.setOnClickListener { viewModel.checkUserGuess() } // onclick przecisku sprawdzającego poprawność podanego słowa
            save.setOnClickListener { viewModel.saveGame() } // zapis gry
            load.setOnClickListener { viewModel.loadGame() } // odczyt gry
            userInput.addTextChangedListener { viewModel.updateUserGuess(it.toString()) } // po zmianie tekstu w polu EditText aktualizujemy pole przechowujące
                                                                                          // aktualne słowo podane przez użytkownika
        }

        return binding.root
    }

    private fun showFinalScoreDialog(score: Int, ) {
        MaterialAlertDialogBuilder(requireContext())
            .setTitle("Gratulacje")
            .setMessage("Wynik: $score")
            .setCancelable(false)
            .setNegativeButton("Wyjdź") { _, _ -> exitGame() } // po wciśnięciu przecisku wychodzimy z aplikacji
            .setPositiveButton("Zagraj ponownie") { _, _ -> viewModel.resetGame() } // reset gry
            .show()
    }

    private fun exitGame(){
        requireActivity().finish() // zamyka aplikację
    }
}

Możemy przetestować grę

<table><tr><td><img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExYWwxOGlnNGQ3Mm9xOGRraHFuaXFyOXA0dXkwNWI2OGx0M2RvbndlMSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c4MVddvPAaRxrwp9JB/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExMnVzODY5dDR1a2QxdDk0MmtibWhmZXM0ZW0yc2VkcWZwd3NkaHlmZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/t3ERZ9ZxBVMeNiY1KY/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExbW8yN201Y3EzOHN6MGZsenhlMTM1ZHU0dWY5bmtkd2lvcTVybnZyZyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/bCYIyjvwkKXnvrGd3n/giphy.gif" width="200" /></td></tr></table>